In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import logging

import numpy as np
import argparse
import copy
import pandas as pd
import shutil
import json
import urllib

from   datetime import datetime, timedelta

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import InvalidRequestError, IntegrityError 
from sqlalchemy.orm.exc import NoResultFound

from matplotlib import pyplot as plt

from tradingdb.utils import *
from tradingdb.ORM.utils import *

logger = logging.getLogger()
logger.setLevel(logging.INFO)
# create file handler which logs even debug messages
handler = logging.StreamHandler()
handler.setLevel(logging.NOTSET)
format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(format)


logger.addHandler(handler)


# get all the FTSE symbols

In [2]:
symbols=download_ftse_symbols('FTSE-100')

In [3]:
len(symbols)

100

In [9]:
symbols.head()

,description,tidm,currency,issuercode,issuername,marketcapitalization,netchangesign,netchange,percentualchange,subSearchTags,lastprice
0,CENTRICA PLC ORD 6 14/81P,CNA,GBX,CECNA,CENTRICA PLC,7449300084,+1,2.30,1.714,None,136.50
1,INTL CONSOLIDATED AIRLINES GROUP SA ORD EUR0.1...,IAG,GBX,INIAG,INTERNATIONAL CONSOLIDATED AIRLINES GROUP S.A.,8117403334,+1,2.25,1.362,None,167.40
2,FRASERS GROUP PLC ORD 10P,FRAS,GBX,SPSPD,FRASERS GROUP PLC,3657607310,+1,10.00,1.251,None,809.50
3,STANDARD CHARTERED PLC ORD USD0.50,STAN,GBX,STGAY,STANDARD CHARTERED PLC,20582491717,+1,5.00,0.678,None,742.60
4,HALEON PLC ORD GBP0.01,HLN,GBX,HALEON,HALEON PLC,31120513810,+1,2.25,0.668,None,339.25


In [5]:
handler.setLevel(logging.DEBUG)
DBNAME=os.path.abspath("../data/master.db")
#if os.path.isfile(DBNAME):
#    os.remove(DBNAME)
#initialize_db(DBNAME,overwrite=True)

def create_all_FTSE100(symbols):

    fh=open("wrong_ftse100_stocks.json","w")
    session=get_new_session(DBNAME)
    with session.begin() as mysession:
        eid=mysession.query(stock.Exchange.id).filter(stock.Exchange.name=="FTSE").one()[0]
        cid=mysession.query(stock.Currency.id).filter(stock.Currency.name=="GBP").one()[0]
        

        for i,st in enumerate(symbols):
            #if i>10: break
            print(i,st)
            
            # get company name from yfinance
            if st[-1]==".":
                symbol=st[:-1]+".L"
            else:   
                symbol=st+".L"
            
            isthere=mysession.query(stock.Stock).filter(stock.Stock.symbol==st).all()
            
            if len(isthere)==0:
                print("Query  YF for symbol",symbol)
                    
                ticker=yf.Ticker(symbol)
                try:
                    info=ticker.info
                except:
                    continue
                if info is None: continue
                if 'longName' in info.keys():
                    name=info['longName']
                elif 'shortName' in info.keys():
                    name=info['shortName']
                else:
                    name=st
                if 'exchange' not in info.keys():
                    print("Missing exchange name")
                    print(ticker)
                    json.dump(info, fh)
                    continue

                exchange=info['exchange']
                if exchange != "LSE":
                    print("Exchanche for %s is %s . Shouldn't it be LSE?"%(ticker,exchange))
                    json.dump(info, fh)
                    continue

                if 'currency' in info.keys():
                    
                    currency=info['currency'].upper()

                    if currency != 'GBP':
                        print("Currency for %s is %s"%(ticker,currency))
                        print("not sure what to do... skipping")
                        
                    else:
                        newstock=stock.Stock(company=name,symbol=st,yf_symbol=symbol,
                                exchange_id=eid,currency_id=cid)    
                        mysession.add(newstock)
                        print("...added!!!")

                else:
                    print("Missing currency!")
                    json.dump(info, fh)
                    continue
            else:
                print("The symbol is already there")

    fh.close()          

create_all_FTSE100(symbols['tidm'].values)



0 IAG
The symbol is already there
1 CNA
The symbol is already there
2 FRAS
The symbol is already there
3 STAN
The symbol is already there
4 HLN
The symbol is already there
5 MRO
The symbol is already there
6 HSBA
The symbol is already there
7 GLEN
The symbol is already there
8 FRES
The symbol is already there
9 RIO
The symbol is already there
10 EXPN
The symbol is already there
11 ENT
The symbol is already there
12 AHT
The symbol is already there
13 CTEC
The symbol is already there
14 AUTO
The symbol is already there
15 AZN
The symbol is already there
16 FLTR
The symbol is already there
17 IHG
The symbol is already there
18 BA.
The symbol is already there
19 BARC
The symbol is already there
20 ULVR
The symbol is already there
21 WTB
The symbol is already there
22 III
The symbol is already there
23 SGRO
The symbol is already there
24 ANTO
The symbol is already there
25 SSE
The symbol is already there
26 MNDI
The symbol is already there
27 SVT
The symbol is already there
28 GSK
The symbo

# add yfinance information to the database

In [7]:
# sample few yfinance and get the headers to create an appropriate table
DBNAME=os.path.abspath("../data/master.db")

session=get_new_session(DBNAME)
ys=[]
with session.begin() as mysession:
    res=mysession.query(stock.Stock).all()
    for i,r in enumerate(res):
        print(i,r.yf_symbol)
        try:
            ys.append(yf.Ticker(r.yf_symbol).info)
        except:
            continue


0 JD.L
1 CCH.L
2 FRES.L
3 SMT.L
4 NXT.L
5 RR.L
6 ADM.L
7 MRO.L
8 SN.L
9 III.L
10 BEZ.L
11 RKT.L
12 HLN.L
13 FCIT.L
14 CNA.L
15 PSH.L
16 INF.L
17 HSV.L
18 TSCO.L
19 WPP.L
20 SMDS.L
21 CRH.L
22 AVV.L
23 MNG.L
24 ABDN.L
25 IMB.L
26 BME.L
27 PSON.L
28 HL.L
29 ANTO.L
30 ULVR.L
31 GSK.L
32 DGE.L
33 IHG.L
34 BARC.L
35 AAL.L
36 PRU.L
37 HSBA.L
38 ENT.L
39 RIO.L
40 KGF.L
41 SBRY.L
42 ITRK.L
43 PHNX.L
44 UTG.L
45 BATS.L
46 BA.L
47 SGE.L
48 NG.L
49 CPG.L
50 BLND.L
51 SMIN.L
52 CTEC.L
53 SSE.L
54 AV.L
55 SKG.L
56 MNDI.L
57 EDV.L
58 SHEL.L
59 NWG.L
60 LAND.L
61 VOD.L
62 ABF.L
63 LSEG.L
64 GLEN.L
65 BRBY.L
66 FLTR.L
67 AZN.L
68 STAN.L
69 LGEN.L
70 BP.L
71 BNZL.L
72 REL.L
73 SGRO.L
74 SPX.L
75 WEIR.L
76 CRDA.L
77 STJ.L
78 WTB.L
79 FRAS.L
80 EXPN.L
81 UU.L
82 RS1.L
83 SVT.L
84 LLOY.L
85 BKG.L
86 IAG.L
87 AHT.L
88 AAF.L
89 RTO.L
90 DCC.L
91 TW.L
92 SDR.L
93 AUTO.L
94 HLMA.L
95 RMV.L
96 OCDO.L
97 PSN.L
98 BDEV.L
99 ^FTSE
100 ^FTAI
101 ^IXIC
102 ^GSPC
103 ^DJI
104 HSX.L
105 JMAT.L
106 IMI.L


In [9]:
handler.setLevel(logging.DEBUG)
#DBNAME=os.path.abspath("../data/master.db")
session=get_new_session(DBNAME)
from datetime import datetime, timedelta

# retrieve all the LSE symbols
deltadays=14
# open/close in UTC 
#UK	8:00 am to 4:30 pm
#US	2:30 pm to 9:00 pm
time_end=datetime.today().replace(hour=4, minute=30, microsecond=0)
# covid crisis: start new era from ftse slump (23 march)
time_start = time_end - timedelta(days=deltadays)
time_start=time_start.replace(hour=8, minute=0, microsecond=0)
print(time_start,time_end)

def add_df_to_yfsymbol(session,symbol,df,interval):
    # get stock id
    with session.begin() as mysession:
            for index, row in df.iterrows():
                # convert name into time
                sid=mysession.query(stock.Stock.id).filter(stock.Stock.yf_symbol==symbol).one()[0]
                pid=mysession.query(stock.Period.id).filter(stock.Period.name==interval).one()[0]
                #print(index)
                q=mysession.query(stock.Price)
                q=q.filter(stock.Price.stock_id==sid)
                q=q.filter(stock.Price.period_id==pid)
                r=q.filter(stock.Price.date==index.to_pydatetime()).one_or_none()
                if r is None:
                    p=stock.Price(
                                stock_id=sid,
                                period_id=pid,
                                date=index.to_pydatetime(),  
                                open=row['Open'],
                                low=row['Low'],
                                high=row['High'],
                                close=row['Close'],
                                adjusted_close=row['Adj Close'],
                                volume=row['Volume'])
                    mysession.add(p)

#[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]          
intervals=['1d','1h','5m']

with session.begin() as mysession:
    q=mysession.query(stock.Stock).join(stock.Exchange).filter(stock.Exchange.name=="FTSE").all()
    symbols=[ r.yf_symbol for r in q ]
    #symbols=symbols[:5]
    print(symbols)
    for interval in intervals:
        dfd=download_data_from_yfsymbols(symbols,time_start,time_end,interval=interval)
        for k,v in dfd.items():
            print(k)# ,v.head())
            add_df_to_yfsymbol(session,k,v,interval)
            #break
        

2023-07-17 08:00:58 2023-07-31 04:30:58
['JD.L', 'CCH.L', 'FRES.L', 'SMT.L', 'NXT.L', 'RR.L', 'ADM.L', 'MRO.L', 'SN.L', 'III.L', 'BEZ.L', 'RKT.L', 'HLN.L', 'FCIT.L', 'CNA.L', 'PSH.L', 'INF.L', 'HSV.L', 'TSCO.L', 'WPP.L', 'SMDS.L', 'CRH.L', 'AVV.L', 'MNG.L', 'ABDN.L', 'IMB.L', 'BME.L', 'PSON.L', 'HL.L', 'ANTO.L', 'ULVR.L', 'GSK.L', 'DGE.L', 'IHG.L', 'BARC.L', 'AAL.L', 'PRU.L', 'HSBA.L', 'ENT.L', 'RIO.L', 'KGF.L', 'SBRY.L', 'ITRK.L', 'PHNX.L', 'UTG.L', 'BATS.L', 'BA.L', 'SGE.L', 'NG.L', 'CPG.L', 'BLND.L', 'SMIN.L', 'CTEC.L', 'SSE.L', 'AV.L', 'SKG.L', 'MNDI.L', 'EDV.L', 'SHEL.L', 'NWG.L', 'LAND.L', 'VOD.L', 'ABF.L', 'LSEG.L', 'GLEN.L', 'BRBY.L', 'FLTR.L', 'AZN.L', 'STAN.L', 'LGEN.L', 'BP.L', 'BNZL.L', 'REL.L', 'SGRO.L', 'SPX.L', 'WEIR.L', 'CRDA.L', 'STJ.L', 'WTB.L', 'FRAS.L', 'EXPN.L', 'UU.L', 'RS1.L', 'SVT.L', 'LLOY.L', 'BKG.L', 'IAG.L', 'AHT.L', 'AAF.L', 'RTO.L', 'DCC.L', 'TW.L', 'SDR.L', 'AUTO.L', 'HLMA.L', 'RMV.L', 'OCDO.L', 'PSN.L', 'BDEV.L', 'HSX.L', 'JMAT.L', 'IMI.L']
[************

yfinance - ERROR - 
2 Failed downloads:
yfinance - ERROR - ['AVV.L', 'HSV.L']: Exception('%ticker%: No data found, symbol may be delisted')



AAF.L
AAL.L
ABDN.L
ABF.L
ADM.L
AHT.L
ANTO.L
AUTO.L
AV.L
AVV.L
AZN.L
BA.L
BARC.L
BATS.L
BDEV.L
BEZ.L
BKG.L
BLND.L
BME.L
BNZL.L
BP.L
BRBY.L
CCH.L
CNA.L
CPG.L
CRDA.L
CRH.L
CTEC.L
DCC.L
DGE.L
EDV.L
ENT.L
EXPN.L
FCIT.L
FLTR.L
FRAS.L
FRES.L
GLEN.L
GSK.L
HL.L
HLMA.L
HLN.L
HSBA.L
HSV.L
HSX.L
IAG.L
IHG.L
III.L
IMB.L
IMI.L
INF.L
ITRK.L
JD.L
JMAT.L
KGF.L
LAND.L
LGEN.L
LLOY.L
LSEG.L
MNDI.L
MNG.L
MRO.L
NG.L
NWG.L
NXT.L
OCDO.L
PHNX.L
PRU.L
PSH.L
PSN.L
PSON.L
REL.L
RIO.L
RKT.L
RMV.L
RR.L
RS1.L
RTO.L
SBRY.L
SDR.L
SGE.L
SGRO.L
SHEL.L
SKG.L
SMDS.L
SMIN.L
SMT.L
SN.L
SPX.L
SSE.L
STAN.L
STJ.L
SVT.L
TSCO.L
TW.L
ULVR.L
UTG.L
UU.L
VOD.L
WEIR.L
WPP.L
WTB.L
[*********************100%***********************]  102 of 102 completed

yfinance - ERROR - 
2 Failed downloads:
yfinance - ERROR - ['HSV.L', 'AVV.L']: Exception('%ticker%: No data found, symbol may be delisted')



AAF.L
AAL.L
ABDN.L
ABF.L
ADM.L
AHT.L
ANTO.L
AUTO.L
AV.L
AVV.L
AZN.L
BA.L
BARC.L
BATS.L
BDEV.L
BEZ.L
BKG.L
BLND.L
BME.L
BNZL.L
BP.L
BRBY.L
CCH.L
CNA.L
CPG.L
CRDA.L
CRH.L
CTEC.L
DCC.L
DGE.L
EDV.L
ENT.L
EXPN.L
FCIT.L
FLTR.L
FRAS.L
FRES.L
GLEN.L
GSK.L
HL.L
HLMA.L
HLN.L
HSBA.L
HSV.L
HSX.L
IAG.L
IHG.L
III.L
IMB.L
IMI.L
INF.L
ITRK.L
JD.L
JMAT.L
KGF.L
LAND.L
LGEN.L
LLOY.L
LSEG.L
MNDI.L
MNG.L
MRO.L
NG.L
NWG.L
NXT.L
OCDO.L
PHNX.L
PRU.L
PSH.L
PSN.L
PSON.L
REL.L
RIO.L
RKT.L
RMV.L
RR.L
RS1.L
RTO.L
SBRY.L
SDR.L
SGE.L
SGRO.L
SHEL.L
SKG.L
SMDS.L
SMIN.L
SMT.L
SN.L
SPX.L
SSE.L
STAN.L
STJ.L
SVT.L
TSCO.L
TW.L
ULVR.L
UTG.L
UU.L
VOD.L
WEIR.L
WPP.L
WTB.L
[*********************100%***********************]  102 of 102 completed

yfinance - ERROR - 
2 Failed downloads:
yfinance - ERROR - ['HSV.L', 'AVV.L']: Exception('%ticker%: No data found, symbol may be delisted')



AAF.L
AAL.L
ABDN.L
ABF.L
ADM.L
AHT.L
ANTO.L
AUTO.L
AV.L
AVV.L
AZN.L
BA.L
BARC.L
BATS.L
BDEV.L
BEZ.L
BKG.L
BLND.L
BME.L
BNZL.L
BP.L
BRBY.L
CCH.L
CNA.L
CPG.L
CRDA.L
CRH.L
CTEC.L
DCC.L
DGE.L
EDV.L
ENT.L
EXPN.L
FCIT.L
FLTR.L
FRAS.L
FRES.L
GLEN.L
GSK.L
HL.L
HLMA.L
HLN.L
HSBA.L
HSV.L
HSX.L
IAG.L
IHG.L
III.L
IMB.L
IMI.L
INF.L
ITRK.L
JD.L
JMAT.L
KGF.L
LAND.L
LGEN.L
LLOY.L
LSEG.L
MNDI.L
MNG.L
MRO.L
NG.L
NWG.L
NXT.L
OCDO.L
PHNX.L
PRU.L
PSH.L
PSN.L
PSON.L
REL.L
RIO.L
RKT.L
RMV.L
RR.L
RS1.L
RTO.L
SBRY.L
SDR.L
SGE.L
SGRO.L
SHEL.L
SKG.L
SMDS.L
SMIN.L
SMT.L
SN.L
SPX.L
SSE.L
STAN.L
STJ.L
SVT.L
TSCO.L
TW.L
ULVR.L
UTG.L
UU.L
VOD.L
WEIR.L
WPP.L
WTB.L
